In [ ]:
from datasets import load_dataset

# a couple of faulty samples creating trouble somewhere between 71400 and 72200, best avoid them 
train_ds=load_dataset('sylvain471/catmus-modern-processed',split='train[:71400]+train[72200:]')
train_ds


In [3]:
import torch
from torch.utils.data import Dataset
# from PIL import Image
from io import BytesIO
import base64
from transformers import TrOCRProcessor
from transformers import VisionEncoderDecoderModel
from transformers import AutoTokenizer

class IMCDataset(Dataset):
    def __init__(self, ds, processor, max_target_length=64):
        self.ds = ds
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        # print(ds['train'][idx])
        text = self.ds[idx]['text']
        image = self.ds[idx]['img_data'].convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        labels = self.processor.tokenizer(text, 
                                          padding="max_length",
                                          truncation=True, 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding
    

# processor = TrOCRProcessor.from_pretrained('medieval-data/trocr-medieval-semitextualis')
# model = VisionEncoderDecoderModel.from_pretrained('medieval-data/trocr-medieval-semitextualis')

tokenizer = AutoTokenizer.from_pretrained("Riksarkivet/trocr-base-handwritten-swe")
processor = TrOCRProcessor.from_pretrained("Riksarkivet/trocr-base-handwritten-swe")
model = VisionEncoderDecoderModel.from_pretrained("Riksarkivet/trocr-base-handwritten-swe")

# processor = TrOCRProcessor.from_pretrained("Riksarkivet/trocr-base-handwritten-swe")
# model = VisionEncoderDecoderModel.from_pretrained("./catmus/checkpoint-3000")

# model = VisionEncoderDecoderModel.from_pretrained('agomberto/trocr-large-handwritten-fr')
# processor = TrOCRProcessor.from_pretrained("sylvain471/troc-medieval-fr-3ch-imc")

train_dataset = IMCDataset(ds=train_ds,processor=processor)
# eval_dataset = IMCDataset(ds=eval_ds,processor=processor)

In [4]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from evaluate import load


# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

cer = load("cer")

def compute_metrics(pred):
    global cer
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer_score = cer.compute(predictions=pred_str, references=label_str)
    print("CER:",cer_score)
    return {"cer": cer_score}


training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    # evaluation_strategy="steps",
    num_train_epochs=1,  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # fp16=True, 
    bf16=True,
    output_dir="./catmus",
    logging_steps=50,
    save_steps=1500,
    # eval_steps=100,
    report_to='tensorboard',
    gradient_accumulation_steps=4
    # resume_from_checkpoint="./catmus/checkpoint-2000"
)

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

trainer.train()


In [ ]:
model.save_pretrained("./catmus")
tokenizer.save_pretrained("./catmus")
processor.save_pretrained("./catmus")

[]

## Push to hub

In [ ]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
token=os.getenv("HF_TOKEN")

repo_name="<your_hf_repo>"

model.push_to_hub(repo_name,token=token)
processor.push_to_hub(repo_name,token=token)
tokenizer.push_to_hub(repo_name,token=token)